# Task 1

Let us now derive the probability of getting to the following table:

|            | Field A1   | Field A2     |            |
|------------|------------|--------------|------------|
| Field B1   | a          | b            | a + b      |
| Field B2   | c          | d            | c + d      |
|            | a + c      | b + d        | N          |

We know that a probability in general can be computed as $p = \frac{\text{favorable outcomes}}{\text{possible outcomes}}$

We will compute the numerator and the denominator separately.
First, we want to see in how many ways we can construct the table by distributing N elements in such a way we get the values a, b, c, d in the table in the way they are presented in the table.
This value is actually $$\binom{N}{a,b,c,d}$$

$$\binom{N}{a,b,c,d} = \binom{N}{a} \binom{N-a}{b} \binom{N-a-b}{c} \binom{N-a-b-c}{d} =
\binom{N}{a} \binom{N-a}{b} \binom{N-a-b}{c} \cdot 1 $$

Where at every step we subtract the ones used in the previous coefficient since we have less elements to distribute.
(We simplified observing that $N-a-b-c = d$)

Then
$$\binom{N}{a,b,c,d} = \frac{N!}{a!(N-a)!} \cdot \frac{(N-a)!}{b!(N-a-b)!} \cdot \frac{(N-a-b)!}{c!(N-a-b-c)!} = \frac{N!}{a!b!c!d!}$$

Now that we have the favorable outcomes, we need to derive all the possible outcomes.
To do so we are going to compute in how many ways we can arrange the N values in such a way that we get $(a+b),(c+d)$ as row sums and $(a+c),(b+d)$ as column sums. This, again, corresponds to computing the binomial coefficient, in this case N over the 2 values for the row sums (and then, column sums).

We will now perform this for the row sums, then it will be the same also for the column sums:
$$\binom{N}{(a+b)(c+d)} = \binom{N}{(a+b)}\binom{N-(a+b)}{(c+d)} = \frac{N!}{(a+b)!(N-(a+b))!} \cdot \frac{(N-a-b)!}{(c+d)!(N-a-b-c-d)!} = \frac{N!}{(a+b)!(c+d)!}$$

We can do the same for the marginal column sums and we get
$$\binom{N}{(a+c),(b+d)} = \frac{N!}{(a+c)!(b+d)!}$$

Now we divide the number of possible the number of the possibilities of obtaining the table a,b,c,d by the number of possiblities of having the marginal row and column sums:

$$ p = \frac{\frac{N!}{a!b!c!d!}}{\frac{N!}{(a+b)!(c+d)!} \cdot \frac{N!}{(a+c)!(b+d)!}} = \frac{(a+b)!(c+d)!(a+c)!(a+d)!}{N!a!b!c!d!} $$

### Before starting

Let us import all the libraries that we will need in the execution of the assignment

In [20]:
import numpy as np
import scipy
from scipy import stats
from tabulate import tabulate

# Task 2

In the following script we are going to compute the probability of getting a table 2 x 2 defined as follows:

|            | Field A1   | Field A2     |
|------------|------------|--------------|
| Field B1   | a          | b            |
| Field B2   | c          | d            |

In [21]:
# Constructs the table and returns a numpy array with its field, the marginal sums, and the total number of observations.
def obtain_table(a,b,c,d):
    # Creating the matrix
    observations = np.array([[a, b], [c, d]])
    # Computing the marginal sums and the overall sum for the matrix
    marginal_col_sums = np.sum(observations, axis=0, keepdims=True)
    marginal_row_sums = np.sum(observations, axis=1, keepdims=True)
    N = np.sum(marginal_col_sums)

    return observations, marginal_col_sums, marginal_row_sums, N


def TabProb(a,b,c,d):
    observations, marginal_col_sums, marginal_row_sums, N = obtain_table(a,b,c,d)

    # Computing now the numerator (in how many ways we can distribute N in such a way to get a,b,c,d in the matrix)
    numerator = 1
    for v in np.nditer(marginal_row_sums):
        numerator *= np.math.factorial(v)
    for v in np.nditer(marginal_col_sums):
        numerator *= np.math.factorial(v)

    # Computing now instead the overall number of outcomes with marginal sums (a+c), (b+d) for the column marginal sums and (a+b),(b+d) for the row sums
    denominator = 1
    for v in np.nditer(observations):
        denominator *= np.math.factorial(v)
    denominator *= np.math.factorial(N)

    # The probability is equal to (Favorable outcomes / overall possibilities)
    probability = numerator / denominator

    return probability

We can now try our method with a simple table for which we can also evaluate the results by hand

|            | Field A1   | Field A2     |
|------------|------------|--------------|
| Field B1   | 1          | 2            |
| Field B2   | 3          | 4            |

The row sums = [3,7], while the column sums = [4,6], respectively.

Let us now compute the formula and we simply get

$$p = \frac{3!7!4!6!}{10!1!2!3!4!} = 0.5$$

Let us now run the code and see what the output actually is

In [22]:
print('p =', TabProb(1,2,3,4))

p = 0.5


Which was what we were expecting, so our method is working fine.

Once that we know that the method is working correctly, we can move ahead by computing the probability of getting the exact table required in the exercise.

In [23]:
print('p =', TabProb(8,1,4,5))

p = 0.06108597285067873


# Task 3

### Answer to the question:
The possible meaningful combinations {$\chi^2$ test, Fisher's test} x {one-sided, two-sided} are:
- $\chi^2$ : one-sided --> The rejection region for $\chi^2$ is on the right (you are looking at the right tail when deciding independence). Moreover, during the computation by taking the square, you forget about the sign of $e_{kl} - a_{kl}$
- Fisher's : one-sided
- Fisher's : two-sided

The table used in our experiment is the following:

|          | Improvement YES | Improvement NO |     |
|----------|-----------------|----------------|-----|
| Drug A   | 8               | 1              | 9   |
| Drug B   | 4               | 5              | 9   |
| ________ | _______________ | ______________ | ___ |
|          | 12              | 6              | 18  |


What we want to see is whether both drugs have the same effect on the patient


## *$\chi^2$ test*

Before starting the test we need to decide the nunll hypothesis.
It will be defined as: $$H_{0} = \text{The improvements for both drugs come from the same distribution}$$

Our alternative hypothesis (to be considered when the null hypothesis is rejected) will be:
$$H_1 = \text{The improvements for the 2 drugs come from different distributions}$$

##### Computation of the statistical $\chi^2$
First, we are going to obtain the table of observations and the expected counts from the given table.
At the end, we will also print the corresponding table.

In [24]:
obs, col_sums, row_sums, n = obtain_table(8,1,4,5)

expected = col_sums * row_sums / n

table = tabulate(expected, tablefmt="fancy_grid")
print(table)

╒═══╤═══╕
│ 6 │ 3 │
├───┼───┤
│ 6 │ 3 │
╘═══╧═══╛


Once we are done with this, we can compute the value for the $\chi^2$ obtained from our distribution of the data

In [25]:
x2stat = np.sum(((obs - expected)**2)/expected)
print('The chi^2 value obtained from our data is =', x2stat)

The chi^2 value obtained from our data is = 4.0


##### Comparison with the $\chi^2$ distribution

When performing the $\chi^2$ test we need to know 2 things:
- The number of degrees of freedom, which are defining the shape of the $\chi^2$ distribution.
    In our case, $dof = (number \hspace{0.2cm} of \hspace{0.2cm} rows - 1) \cdot (number \hspace{0.2cm} of \hspace{0.2cm} columns - 1)$
- The level of significance $\alpha$, which in this case was given from the text of the exercise: $\alpha = 0.05$

In [26]:
dof = (obs.shape[0] - 1) * (obs.shape[1] - 1)
alpha = 0.05

print('dof =',dof)
print('alpha =', alpha)

dof = 1
alpha = 0.05


Now we want to compute $\chi^2$ such that $$1 - cdf(\chi^2) = \alpha \iff sf(\chi^2) = \alpha \iff \chi^2 = isf(\alpha)$$

Where:
 - cdf = comulative density function
- sf = survival function $(1 - cdf)$
- isf = inverse survival function

In [27]:
distr_chi2 = scipy.stats.chi2.isf(alpha, df=dof)
print('The \'boundary\' chi^2 from the distribution is:', distr_chi2)

The 'boundary' chi^2 from the distribution is: 3.8414588206941285


Now we can perform the actual test since we have both $\chi^2_{stat}$ and $\chi^2_{1}(\alpha)$.

If $\chi^2_{stat} \geq \chi^2_{1}(\alpha)$, then we can reject the null hypothesis.
Let us now perform the aforementioned test

In [28]:
if x2stat >= distr_chi2:
    print('The null hypothesis is rejected')
else:
    print('The null hypothesis can\'t be rejected')

The null hypothesis is rejected


## *Fisher's test*

The Fisher's test is usually used when we need to state the independence between 2 variables and our contingency table is a 2x2 table.
It is usually used when the expected frequencies are lower than 5 and therefore the chi square test wouldn't give a reliable estimate.

#### One-sided (less)
Before performing the test it is *very important* to formulate correctly the null-hypothesis.
When using a one-sided test, we are assuming that only one side is relevant to us (and we can do this thanks to our previous knowledge of the problem) and we are aware of the consequence of missing the effect in the other (untested) direction.

In this case, we want to see if Drug A performs better than Drug B or not.
Therefore, our null hypothesis will be "Drug A is no better than Drug B"
Which can be reformulated as
$$H_0 = \text{improvements due to Drug A \leq improvements due to Drug B}$$

The alternative hypothesis instead will be:
$$H_1 = \text{improvements due to Drug A > improvements due to Drug B }$$

If our test will reject the null hypothesis, then this means that the Drug A works actually better than Drug B.

However, keep in mind that when doing this we are totally forgetting about the possibility of having an effect in the opposite direction.
This can be useful sometimes, however less robust than a two-tailed test.


The probability of getting the given table or something even further away from the expected value is defined as:

$$P = \Sigma_{i=0}^{a}\frac{(a+b)!(c+d)!(a+c)!(b+d)!}{n!(a-i)!(b+i)!(c+i)!(d-i)!}$$

Under the assumption that the value in the top-left corner is the minimum value of the observations in the contingency table.
In order to achieve this, we can do a permutation of the columns in such a way to transform this table:

|                 | Improvement YES       | Improvement NO         |        |
|-----------------|-----------------------|------------------------|--------|
| Drug A          | 8                     | 1                      | 9      |
| Drug B          | 4                     | 5                      | 9      |
| ____________ | _____________________ | _____________________ | ______ |
|                 | 12                    | 6                      | 18     |

into this one:

|          | Improvement NO  | Improvement YES |     |
|----------|-----------------|-----------------|-----|
| Drug A   | 1               | 8               | 9   |
| Drug B   | 5               | 4               | 9   |
| ____________ | _____________________ | _____________________ | ______ |
|          | 6               | 12              | 18  |


**NOTE:**
Since the probability of the table does not change when we do a permutation of the rows or of the columns, we could as well leave the table untouched and increase the top-left value up to 9 (so only 2 tables will be considered anyway).
I used this approach so it is not different from what we have seen during the theoretical lecture.

In [29]:
tmp = obs.copy()
tmp[:,[0,1]] = tmp[:,[1,0]]

table = tabulate(tmp, tablefmt="fancy_grid")
print('The permuted table is ')
print(table)

The permuted table is 
╒═══╤═══╕
│ 1 │ 8 │
├───┼───┤
│ 5 │ 4 │
╘═══╧═══╛


In [30]:
def compute_prob_equal_or_further(data):
    overall_prob = 0
    while data[0,0] >= 0:
        table = tabulate(data, tablefmt="fancy_grid")

        prob = TabProb(data[0,0], data[0,1], data[1,0], data[1,1])
        overall_prob += prob

        print('The probability of having table')
        print(table)
        print('is p =', prob, '\n')

        data[0,0] -= 1
        data[1,1] -= 1
        data[0,1] += 1
        data[1,0] += 1

    return overall_prob

overall_prob_left = compute_prob_equal_or_further(tmp)
print('The overall probability is =', overall_prob_left)

The probability of having table
╒═══╤═══╕
│ 1 │ 8 │
├───┼───┤
│ 5 │ 4 │
╘═══╧═══╛
is p = 0.06108597285067873 

The probability of having table
╒═══╤═══╕
│ 0 │ 9 │
├───┼───┤
│ 6 │ 3 │
╘═══╧═══╛
is p = 0.004524886877828055 

The overall probability is = 0.06561085972850679


The $\alpha$ level of confidence is always $\alpha=0.05$
If $P < alpha$, then the null hypothesis is rejected

In [31]:
alpha = 0.05

if overall_prob_left < alpha:
    print('The null hypothesis is rejected')
else:
    print('The null hypothesis is not rejected')

The null hypothesis is not rejected


### One-sided (greater):

As before, we need to re-state what the null hypothesis is.
In this case instead we will consider whether Drug A will perform worse than Drug B.
Therefore, we need to define the null hypothesis as:
$$H_0 = \text{improvements due to Drug A \geq improvements due to Drug B}$$

The alternative hypothesis will be instead:
$$H_1 = \text{improvements due to Drug A < improvements due to Drug B}$$

Thus, if we reject the null hypothesis we can say that it is likely (how much likely depends on the significance level chosen) that Drug A will be worse than Drug B.

Before we were on the left side with respect to the expected value, so now we need to move to the right.
In order to do so, we will sum the expected values of the first cell to the values along the main diagonal and subtract it to the ones on secondary diagonal (we need to do this in order to preserve the row and column sums).

Once done that, we can proceed by increasing the value in the top-left until it is equal to the row sum of the respective row.

In [32]:
tmp, col_sums, row_sums, n = obtain_table(1,8,5,4)
expected = col_sums * row_sums / n

tmp[0,0] = tmp[0,0] + (expected[0,0] + 1)
tmp[0,1] = tmp[0,1] - (expected[0,0] + 1)
tmp[1,0] = tmp[1,0] - (expected[0,0] + 1)
tmp[1,1] = tmp[1,1] + (expected[0,0] + 1)

We can now see how our matrix looks like:

In [33]:
table = tabulate(tmp, tablefmt="fancy_grid")
print('The table on the other side with respect to the expected value is')
print(table)

The table on the other side with respect to the expected value is
╒═══╤═══╕
│ 5 │ 4 │
├───┼───┤
│ 1 │ 8 │
╘═══╧═══╛


As we can see, the row sums and column sums are preserved.
In a way similar to what we did before, we will perform a permutation of the rows in such a way that the minimum value is in the top left corner.
Again, this will not influence the probability of obtaining such a table

In [34]:
tmp[[0,1]] = tmp[[1,0]]
table = tabulate(tmp, tablefmt="fancy_grid")
print('The table after the permutation is')
print(table)

The table after the permutation is
╒═══╤═══╕
│ 1 │ 8 │
├───┼───┤
│ 5 │ 4 │
╘═══╧═══╛


We can now compute the probability of such a table or even further away from the expected value with the same function defined above

In [35]:
overall_prob_right = compute_prob_equal_or_further(tmp)
print('The overall probability to the right is =', overall_prob_right)

The probability of having table
╒═══╤═══╕
│ 1 │ 8 │
├───┼───┤
│ 5 │ 4 │
╘═══╧═══╛
is p = 0.06108597285067873 

The probability of having table
╒═══╤═══╕
│ 0 │ 9 │
├───┼───┤
│ 6 │ 3 │
╘═══╧═══╛
is p = 0.004524886877828055 

The overall probability to the right is = 0.06561085972850679


#### Note

We can observe that the result we obtained for the "greater" and "less" tails are the same.
In fact, in the exact Fisher's test the probability of the contingency table on one side and on the other of the expected value is the same.

### Fisher's test (two-sided)
We will look now at the two-sided version of the Fisher's test.

This kind of test is more robust generally and we do not care about which drug is more effective than the other, we are just interested in any kind of correlation between the two.
Our null hypothesis will be:
$$H_0 = \text{Both drugs come from the same distribution}$$

And, in the same way as before, the alternative hypothesis is:
$$H_1 = \text{The 2 drugs come from 2 different distributions}$$

Since we already have the p-values for the left-sided and right-sided fisher's test, we can just sum them up in order to obtain the 2-sided p-value.

In [36]:
fisher_two_sided = overall_prob_right + overall_prob_left
print(fisher_two_sided)

if fisher_two_sided >= alpha:
    print('The null hypothesis can\'t be rejected')
else:
    print('The null hypothesis is rejected')

0.13122171945701358
The null hypothesis can't be rejected


## Comparison with the scipy library functions

We are now going to compute the comparison of this with the scipy library function 'scipy.stats.fisher_exact()' and 'scipy.stats.chi2_contingency()'

In [37]:
obs, col_sums, row_sums, n = obtain_table(1,8,5,4)
print('The p-value of the one-tailed (to the left) Fisher\'s test by using the library is:')
print(scipy.stats.fisher_exact(obs, alternative='less')[1])

expected = col_sums * row_sums / n
tmp = obs.copy()
tmp[0,0] = tmp[0,0] + (expected[0,0] + 1)
tmp[0,1] = tmp[0,1] - (expected[0,0] + 1)
tmp[1,0] = tmp[1,0] - (expected[0,0] + 1)
tmp[1,1] = tmp[1,1] + (expected[0,0] + 1)
print('The p-value of the one-tailed (to the right) Fisher\'s test by using the library is:')
print(scipy.stats.fisher_exact(tmp, alternative='greater')[1])

print('The p-value of the two-tailed Fisher\'s test by using the library is:')
print(scipy.stats.fisher_exact(obs, alternative='two-sided')[1])

The p-value of the one-tailed (to the left) Fisher's test by using the library is:
0.06561085972850689
The p-value of the one-tailed (to the right) Fisher's test by using the library is:
0.0656108597285069
The p-value of the two-tailed Fisher's test by using the library is:
0.13122171945701377


In [38]:
print('The p-value of the one-tailed chi square test by using the library is:')
# We set the correction=False since the default value is True: is the degree of freedoms are 1, then the Yates' correction for continuity is applied
print(scipy.stats.chi2_contingency(obs, correction=False)[1])

The p-value of the one-tailed chi square test by using the library is:
0.04550026389635857


## Observations

One interesting thing can be observed from our results:
While the $\chi^2$ test was rejecting the null-hypothesis, this wasn't true when using the Fisher's test (one-sided).
The more reliable result is the one we obtained from the *Exact Fisher's test*.
We shouldn't look at the results obtained with the $\chi^2$ test since we are using it against the assumption that all the expected values must be $\geq5$. In fact, $\chi^2$ distribution is a continuous distribution and in order it you need enough samples, otherwise the approximation will not be exact.